In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid,KarateClub

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
dataset = KarateClub()
data = dataset[0]
data

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [7]:
# 定义GNN模型
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [11]:
# 初始化模型
model = GCN(in_channels=data.num_node_features, hidden_channels=16, out_channels=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# 训练模型（使用所有节点）
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)  # 注意这里不再用 train_mask
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        pred = out.argmax(dim=1)
        acc = (pred == data.y).sum().item() / data.num_nodes
        print(f"Epoch {epoch} - Loss: {loss.item():.4f} - Acc: {acc:.4f}")


Epoch 0 - Loss: 1.3731 - Acc: 0.3235
Epoch 20 - Loss: 0.8248 - Acc: 0.7941
Epoch 40 - Loss: 0.3297 - Acc: 0.9412
Epoch 60 - Loss: 0.1338 - Acc: 1.0000
Epoch 80 - Loss: 0.0728 - Acc: 1.0000
Epoch 100 - Loss: 0.0471 - Acc: 1.0000
Epoch 120 - Loss: 0.0347 - Acc: 1.0000
Epoch 140 - Loss: 0.0279 - Acc: 1.0000
Epoch 160 - Loss: 0.0239 - Acc: 1.0000
Epoch 180 - Loss: 0.0213 - Acc: 1.0000


In [14]:
from sklearn.model_selection import train_test_split

# 手动划分 train/test
num_nodes = data.num_nodes
train_idx, test_idx = train_test_split(torch.arange(num_nodes), test_size=0.3, random_state=42)

data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_idx] = True
data.test_mask[test_idx] = True

# 初始化模型
model = GCN(in_channels=data.num_node_features, hidden_channels=16, out_channels=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# 训练只用训练节点
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        pred = out.argmax(dim=1)
        acc = (pred[data.train_mask] == data.y[data.train_mask]).float().mean().item()
        print(f"Epoch {epoch} - Loss: {loss.item():.4f} - Train Acc: {acc:.4f}")


Epoch 0 - Loss: 1.3741 - Train Acc: 0.4783
Epoch 20 - Loss: 0.8977 - Train Acc: 0.8696
Epoch 40 - Loss: 0.4276 - Train Acc: 0.8696
Epoch 60 - Loss: 0.1713 - Train Acc: 1.0000
Epoch 80 - Loss: 0.0698 - Train Acc: 1.0000
Epoch 100 - Loss: 0.0370 - Train Acc: 1.0000
Epoch 120 - Loss: 0.0253 - Train Acc: 1.0000
Epoch 140 - Loss: 0.0202 - Train Acc: 1.0000
Epoch 160 - Loss: 0.0174 - Train Acc: 1.0000
Epoch 180 - Loss: 0.0158 - Train Acc: 1.0000


In [15]:
model.eval()
out = model(data)
pred = out.argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean().item()
print(f"Test Accuracy: {acc:.4f}")


Test Accuracy: 0.9091


当然可以！我们来把图神经网络（GNN）在 `KarateClub` 上的训练过程，用一个**故事 + 可视化心智图**的方式讲清楚，就像是“村民互相交流”的社交过程——既生动，又不失技术细节 🧠💡

---

## 🎬 故事背景：空手道俱乐部的秘密聚会

想象我们现在是「空手道俱乐部」的一群成员，这个俱乐部的结构是一张图，节点是人，边是“谁和谁熟”。现在我们想做一件事：

> 🕵️‍♀️**我们要预测每个人是属于红队还是蓝队！**

我们知道一些人的队伍（有标签），但更多人我们不知道（无标签）。于是我们要让模型自己「猜」出每个人属于哪个队！

---

## 🧠 GCN 训练过程 = 图上的“八卦传播 + 自我认知”

我们把整个 GCN 的训练过程比喻成 3 个阶段：

---

### 🚶 第一步：初始化（每人都有一点“自我认知”）

每个节点（人）都有自己的特征向量，比如说：
- 我是第几号会员（one-hot 编码）；
- 我的活动活跃度（degree）；
- 我的朋友圈人数…

这就像是每个人脑袋里都有个小本本，记录了“我是谁”。

---

### 📣 第二步：**八卦传播（图卷积）**

#### 类比：
> 每一轮训练，就像每个会员晚上聚在一起闲聊：“你昨天和谁练过？”、“你是红队的吗？”

模型中对应的就是：
- 每个节点 **收集邻居的特征信息**（message passing）
- 然后把邻居说的内容 + 自己的想法合并（aggregation + update）

🔄 每进行一层 GCN，信息就传播一圈（图的 hops），每个人了解得更多，对自己判断也更有信心了！

---

### 🧪 第三步：计算误差 + 学习（梯度下降）

我们告诉模型一些人真实是“红队/蓝队”，就像告诉几个村民：
> “你们知道队长是谁，他是红队！”

然后模型会做以下事：
1. 对所有有标签的人，**模型的预测和真实值对比（误差）**；
2. 根据误差调整“每个节点脑袋里的本本”（权重更新）；
3. 继续下一轮“八卦传播”，但信息变得越来越有指导意义！

这样一轮一轮训练下去，模型越来越懂得如何从图结构中判断每个人该属于哪个队伍。

---

## 🔁 一句话总结 GCN 的训练过程：

> 每个人开始都不知道自己是谁，通过不断地听邻居的建议（信息聚合），结合自己的判断，不断更新认知（特征更新），最终大家都能准确判断自己的身份（节点分类）。

---

## 🎨 可视化示意图（思维导图）

```
图结构（Karate Club 图）
   |
初始化特征（节点 -> 向量）
   |
图卷积层 1（邻居信息聚合）
   |
ReLU 激活
   |
图卷积层 2（再次聚合）
   |
LogSoftmax 分类概率
   |
NLLLoss（和真实标签对比）
   |
反向传播（调整权重）
   |
重复200轮训练
   ↓
准确预测每个节点的类别
```

---

如果你喜欢，我可以画成图给你展示整张流程图或者模拟不同颜色传播效果！现在你有没有更清晰地理解整个过程？需要我把它转成 Markdown 图解或图片图示吗？🎨🧩

这个问题问得非常好！我们来拆解一下：

---

## 🧩 整体框架回顾一下：

在 GNN 的训练中（比如 GCN），我们训练的是一个**图结构的数据集**，所以输入不仅仅是一个数组，而是图+特征+标签的组合：

---

## ✅ 1️⃣ 输入数据由哪些部分组成？

在 PyTorch Geometric 中，图数据通常封装在一个 `Data` 对象中，主要包括：

| 字段 | 含义 | 示例维度 |
|------|------|----------|
| `x` | 节点特征矩阵 | `[num_nodes, num_node_features]` |
| `edge_index` | 边的连接（图的结构） | `[2, num_edges]` |
| `y` | 节点的标签（分类目标） | `[num_nodes]` |
| `train_mask` / `test_mask` | 标记哪些节点参与训练 / 测试 | `[num_nodes]` 布尔向量 |

### 🌰 例如：Karate Club

```python
data.x.shape       # [34, 34]  每个节点是 one-hot 特征
data.edge_index    # [2, 156]  表示 78 条边
data.y             # [34]      每个节点的类别标签（0 或 1）
```

---

## ✅ 2️⃣ 标签（`y`）一般是什么样子的？

### 🎯 节点分类任务中：
- `data.y[i] = 0` 表示第 `i` 个节点是红队
- `data.y[i] = 1` 表示第 `i` 个节点是蓝队
- 如果有更多类别，比如 3 类，那就是 `[0, 1, 2]`

### 🧠 多数情况下，`y` 是一个**整型张量**，不需要 One-hot，比如：

```python
data.y = tensor([0, 1, 0, 2, 1, 1, ...])  # 每个节点的类别
```

> 🗒️ 在计算 Loss 前，模型输出会是 `log_softmax` 结果，和 `y` 里的“整型标签”结合计算 loss。

---

## ✅ 3️⃣ GNN 中 loss 是怎么计算的？

我们用的是 `F.nll_loss()`（负对数似然损失），是用于 **分类任务** 的标准方法。

假设：
```python
out = model(data)        # shape: [num_nodes, num_classes]
                         # 每个节点对每个类别的 log 概率
```

那么 loss 计算是：
```python
loss = F.nll_loss(out[train_mask], data.y[train_mask])
```

**解释一下这句话的含义：**

| 步骤 | 含义 |
|------|------|
| `out` | 模型输出的 **log 概率**（用 `log_softmax` 输出） |
| `data.y` | 每个节点的真实标签（整数） |
| `train_mask` | 我们只对训练集里的节点计算损失 |
| `nll_loss` | 负对数似然损失（会自动查找标签对应位置的概率） |

### 📌 举个例子

假设 `out[3] = [-1.2, -0.1, -2.0]` 表示第 3 个节点对 3 类的 log 概率，而真实标签是 `data.y[3] = 1`，那么损失就是：

\[
\text{Loss}_3 = - \log(P_3^{(1)}) = -(-0.1) = 0.1
\]

---

## ✅ 总结一句话

> GNN 模型的输入是一个图结构（边）+ 节点特征（x）+ 节点标签（y），训练目标是通过邻居的信息预测每个节点的类别，通过 log-softmax 输出概率，用负对数似然损失（NLLLoss）和真实标签做对比学习。

---

如果你想看看实际 `x`、`y`、`edge_index` 的样子，我可以帮你打印出来几行；或者你想我给出完整的 loss 流程图也可以！

你想深入理解哪一部分？比如：
- 图结构如何参与 message passing？
- 多分类和二分类有何不同？
- 标签缺失怎么办？

我可以继续拆解！